# CAPSTONE PROJECT - Genetic Variant Classification


Background: 
ClinVar is a public archive of interpretations of clinically relevant variants where many researchers can share their variant interpretations. There has been over a million submissions so far. Sometimes, some of these submissions lead to some variants to have conflicting classifications such as when one lab says a variant is benign and another says it is pathogenic. 

Here are some of the recent articles and publications about the conflicting variants in Clinvar: 

https://www.precisiononcologynews.com/cancer/brca2-variants-unknown-significance-reclassified-through-functional-data-additions#.YUOR5Z5KiVY

https://www.nature.com/articles/s41598-019-57335-5

https://www.healthcareitnews.com/news/teens-precision-medicine-analytics-website-highlights-value-data-democratization

http://variantexplorer.org/

https://www.ncbi.nlm.nih.gov/clinvar/docs/faq/

https://f1000researchdata.s3.amazonaws.com/manuscripts/15752/5ca368c4-e377-47f8-9cc5-28053692872e_14470_-_robert_butler.pdf?doi=10.12688/f1000research.14470.1&numberOfBrowsableCollections=26&numberOfBrowsableInstitutionalCollections=4&numberOfBrowsableGateways=29

https://www.genomeweb.com/molecular-diagnostics/clingen-implementing-strategies-resolve-variant-classification-conflicts?utm_source=TrendMD&utm_medium=TrendMD&utm_campaign=0&trendmd-shared=0#.YUOT455KiVY

https://genomemedicine.biomedcentral.com/articles/10.1186/s13073-019-0688-9

https://www.sciencedirect.com/science/article/pii/S0002929718300879


https://www.genomeweb.com/molecular-diagnostics/more-million-records-clinvar-value-grows-variant-classification-resource#.YUOTkJ5KiVY

https://www.genomeweb.com/molecular-diagnostics/clingen-implementing-strategies-resolve-variant-classification-conflicts?utm_source=TrendMD&utm_medium=TrendMD&utm_campaign=0&trendmd-shared=0#.YUOT455KiVY

https://www.genomeweb.com/clinical-genomics/tackling-vus-challenge-are-public-databases-solution-or-liability-labs?utm_source=TrendMD&utm_medium=TrendMD&utm_campaign=0&trendmd-shared=0#.YUOT7J5KiVY


https://ascopubs.org/doi/10.1200/JCO.2016.68.4316
https://genomemedicine.biomedcentral.com/articles/10.1186/s13073-019-0688-9





# Problem definition:


In [ ]:
This problem had been published and explained on kaggle: https://www.kaggle.com/kevinarvai/clinvar-conflicting  :
        
CLINVAR Genetic Variant Classification: Prediction of whether a variant will have conflicting clinical 
classifications. CLINVAR is a public resource containing annotations about human genetic variants. 
These variants are (usually manually) classified by clinical laboratories on a categorical spectrum ranging
from benign, likely benign, uncertain significance, likely pathogenic, and pathogenic. 
Variants that have conflicting classifications (from laboratory to laboratory) can cause confusion when
clinicians or researchers try to interpret whether the variant has an impact on the disease of a given patient. 
https://www.ncbi.nlm.nih.gov/clinvar/


# Objective:


In [ ]:

To predict whether a CLINVAR variant will have conflicting classifications.  
A binary classification problem, where each record in the dataset is a genetic variant with many biological 
and clinical features. 


# What I’m building:


In [ ]:

I downloaded the CLINVAR dataset that was published on Kaggle website. 
I am building a few different models by training various types of binary classifiers that will predict 
whether a variant will have conflicting clinical classifications or not. 



# How I’m using what I learned at Springboard:


In [ ]:

I learned about various Data wrangling methods which I am applying to the dataset to extract important features
using Python and the Pandas library.  

I also learned about supervised learning methods that require splitting data into training and testing datasets and
doing cross-validation to find the best parameters of a classifier to increase its performance. 
I am using multiple binary classifiers, such as logistic regression,random forests, XGBoost, SVM and
tuning parameters which is an important step to build models that fit the data.
This requires trial and error which I am applying in my capstone project as I am trying to solve this problem.

# RESULTS SUMMARY:


In [ ]:
During this project, I had done a lot of Exploratory Data Analysis (EDA) and ended up training many models with 
different parameters as I was tuning the models. In this notebook, I am listing 4 of these models with their final
parameters and best models exported. The best results were almost always achived by XGBoost with the highest ROC being 0.76. 
Random forest was 0.74 whereas Logistic regression and SVM didn't perform well and results were random. 


# MODEL DEPLOYMENT ON STREAMLIT:


In [ ]:
I had deployed these 4 models using streamlit where a user can input a file with variants and get prediction results 
on if a variant is a conflicting variant or not. I had hosted the streamlit app that I had build in the following link:

STREAMLIT LINK:
https://share.streamlit.io/gulsahaltun/mlcapstoneproject/main.py 

# CODE:


In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sklearn.model_selection
import category_encoders as ce

Load training dataset:

In [2]:
df = pd.read_csv("./TRAINING_DATA_FINAL.csv")
df.head()


,CHROM,POS,AF_ESP,AF_EXAC,AF_TGP,CLASS,STRAND,CADD_PHRED,CADD_RAW
0,1,1168180,0.0771,0.10020,0.1066,0,1.0,1.053,-0.208682
1,1,1470752,0.0000,0.00000,0.0000,0,-1.0,31.000,6.517838
2,1,1737942,0.0000,0.00001,0.0000,1,-1.0,28.100,6.061752
3,1,2160305,0.0000,0.00000,0.0000,0,1.0,22.500,3.114491
4,1,2160305,0.0000,0.00000,0.0000,0,1.0,24.700,4.766224


In [3]:
df.shape

(65176, 9)

In [4]:
df.CLASS.value_counts()


0    48747
1    16429
Name: CLASS, dtype: int64

In [5]:
pd.DataFrame([[i, len(df[i].unique())] for i in df.columns], columns=['Columns', 'Unique']).set_index('Columns')

,Unique
Columns,
CHROM,23
POS,63103
AF_ESP,2842
AF_EXAC,6663
AF_TGP,2087
CLASS,2
STRAND,3
CADD_PHRED,9325
CADD_RAW,63792


In [6]:
numerics = df.select_dtypes(exclude=object) 

In [7]:
numerics.head()

,CHROM,POS,AF_ESP,AF_EXAC,AF_TGP,CLASS,STRAND,CADD_PHRED,CADD_RAW
0,1,1168180,0.0771,0.10020,0.1066,0,1.0,1.053,-0.208682
1,1,1470752,0.0000,0.00000,0.0000,0,-1.0,31.000,6.517838
2,1,1737942,0.0000,0.00001,0.0000,1,-1.0,28.100,6.061752
3,1,2160305,0.0000,0.00000,0.0000,0,1.0,22.500,3.114491
4,1,2160305,0.0000,0.00000,0.0000,0,1.0,24.700,4.766224


In [8]:
numerics.shape

(65176, 9)

In [9]:
cleaned2=numerics

In [10]:
cleaned2.head()

,CHROM,POS,AF_ESP,AF_EXAC,AF_TGP,CLASS,STRAND,CADD_PHRED,CADD_RAW
0,1,1168180,0.0771,0.10020,0.1066,0,1.0,1.053,-0.208682
1,1,1470752,0.0000,0.00000,0.0000,0,-1.0,31.000,6.517838
2,1,1737942,0.0000,0.00001,0.0000,1,-1.0,28.100,6.061752
3,1,2160305,0.0000,0.00000,0.0000,0,1.0,22.500,3.114491
4,1,2160305,0.0000,0.00000,0.0000,0,1.0,24.700,4.766224


In [11]:
cleaned2.isnull().sum()


CHROM            0
POS              0
AF_ESP           0
AF_EXAC          0
AF_TGP           0
CLASS            0
STRAND          14
CADD_PHRED    1092
CADD_RAW      1092
dtype: int64

In [12]:
final = cleaned2.dropna()

In [13]:
final.isnull().sum()


CHROM         0
POS           0
AF_ESP        0
AF_EXAC       0
AF_TGP        0
CLASS         0
STRAND        0
CADD_PHRED    0
CADD_RAW      0
dtype: int64

First, we try a basic Logistic Regression:

* Split the data into a training and test (hold-out) set
* Train on the training set, and test for accuracy on the testing set

# LOGISTIC REGRESSION


In [28]:

import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from joblib import dump, load

X_train, X_test, y_train, y_test = train_test_split(final.drop('CLASS',  axis=1), final.CLASS)

# Standard logistic regression
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)



In [29]:
y_pred_lr

array([0, 0, 0, ..., 0, 0, 0])

In [30]:
X_test.head()

,CHROM,POS,AF_ESP,AF_EXAC,AF_TGP,STRAND,CADD_PHRED,CADD_RAW
62332,22,29083950,0.0001,0.00003,0.0000,-1.0,23.900,4.199662
27763,9,6610329,0.0018,0.00000,0.0004,-1.0,7.039,0.450299
22786,7,117232481,0.0017,0.00195,0.0008,1.0,0.095,-0.648802
54473,17,59761349,0.0000,0.00000,0.0000,-1.0,0.004,-1.328957
19085,5,112178923,0.0000,0.00001,0.0000,1.0,0.331,-0.428094


In [31]:
### Logistic Regression results:

In [32]:
roc_auc_score(y_test, lr.predict_proba(X_test)[:,1]) 

0.4971431782411376

In [33]:
roc_auc_score(y_test, lr.predict_proba(X_test)[:,1]) 

0.4971431782411376

In [34]:
lr.predict_proba(X_test)[:,1]

array([0.4344947 , 0.48503007, 0.25687613, ..., 0.47487989, 0.41334492,
       0.35123532])

In [35]:
lr.predict_proba(X_test)[:,0]

array([0.5655053 , 0.51496993, 0.74312387, ..., 0.52512011, 0.58665508,
       0.64876468])

In [38]:
dump(lr, 'Model_logreg.joblib') 
clf_loaded_LogReg = load('Model_logreg.joblib') 

roc_auc_score(y_test, clf_loaded_LogReg.predict_proba(X_test)[:,1]) 


0.4971431782411376

In [ ]:
As can be seen from the results above, the logistic regression gave poor results. 

# RANDOM FOREST


In [ ]:
#Random search: 

In [39]:
from joblib import dump, load
from sklearn.model_selection import RandomizedSearchCV



modelrandom = RandomForestClassifier( random_state=0)

param_grid_random = { 
    'n_estimators': [10, 25, 100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [1,2,3,4,5,6,7,8,9,10],
    'criterion' :['gini', 'entropy']
}

randomsearch = RandomizedSearchCV(estimator=modelrandom, param_distributions=param_grid_random, cv = 2, scoring = "roc_auc", n_iter = 10)


# Fit the model
randomsearch.fit(X_train, y_train)


best_model = randomsearch.best_estimator_

roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1]) 



0.742304201260259

In [53]:
#save the model
dump(best_model, 'BestModelRANDOMFOREST.joblib') 
clf_loaded = load('BestModelRANDOMFOREST.joblib') 

roc_auc_score(y_test, clf_loaded.predict_proba(X_test)[:,1]) 


0.742304201260259

In [54]:
clf_loaded.predict_proba(X_test)[:,0]

array([0.77102979, 0.52227961, 0.54343481, ..., 0.80746952, 0.50900445,
       0.8079005 ])

In [57]:
clf_loaded.predict_proba(X_test)[:,1]

array([0.22897021, 0.47772039, 0.45656519, ..., 0.19253048, 0.49099555,
       0.1920995 ])

# XGBOOST


In [56]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

#model = RandomForestClassifier(n_estimators=100, random_state=0)
modelxgboost = XGBClassifier(random_state=0)

# A parameter grid for XGBoost
param_grid_random = { 
    'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
}

randomsearch3 = RandomizedSearchCV(estimator=modelxgboost, param_distributions=param_grid_random, cv = 2, scoring = "roc_auc", n_iter = 10)

randomsearch3.fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:51:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:51:24] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:51:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:51:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=0, reg_alpha=None,
                             

In [58]:
best_model2 = randomsearch3.best_estimator_

In [59]:
roc_auc_score(y_test, best_model2.predict_proba(X_test)[:,1]) 

0.756235008724033

In [60]:
from joblib import dump, load

#best_model = search.best_estimator_
best_model1 = randomsearch3.best_estimator_

#roc_auc_score(y_test, model.predict_proba(X_test)[:,1]) 
roc_auc_score(y_test, best_model1.predict_proba(X_test)[:,1]) 


#roc_auc_score(y_test, model.predict_proba(X_test)[:,1]) 
best_model1.predict_proba(X_test)[:,1] 

#roc_auc_score(y_test, model.predict_proba(X_test)[:,1]) 
best_model1.predict_proba(X_test)[:,0]


dump(best_model1, 'BestModelXGBOOST3.joblib') 
clf_loaded2 = load('BestModelXGBOOST3.joblib') 

roc_auc_score(y_test, clf_loaded2.predict_proba(X_test)[:,1]) 


0.756235008724033

In [63]:
#roc_auc_score(y_test, model.predict_proba(X_test)[:,1]) 
roc_auc_score(y_test, best_model1.predict_proba(X_test)[:,1]) 

0.756235008724033

In [64]:
#roc_auc_score(y_test, model.predict_proba(X_test)[:,1]) 
best_model1.predict_proba(X_test)[:,1] 

array([0.16735247, 0.5782301 , 0.535776  , ..., 0.0856683 , 0.51513803,
       0.22697796], dtype=float32)

In [65]:
#roc_auc_score(y_test, model.predict_proba(X_test)[:,1]) 
best_model1.predict_proba(X_test)[:,0] 

array([0.83264756, 0.42176992, 0.46422398, ..., 0.9143317 , 0.48486197,
       0.77302206], dtype=float32)

In [ ]:
roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1]) 

In [66]:
#roc_auc_score(y_test, model.predict_proba(X_test)[:,1]) 
best_model1.predict_proba(X_test) 

array([[0.83264756, 0.16735247],
       [0.42176992, 0.5782301 ],
       [0.46422398, 0.535776  ],
       ...,
       [0.9143317 , 0.0856683 ],
       [0.48486197, 0.51513803],
       [0.77302206, 0.22697796]], dtype=float32)

In [67]:
best_model1.predict(X_test) 

array([0, 1, 1, ..., 0, 1, 0])

# SVM


In [62]:
from joblib import dump, load

from sklearn import svm

clf = svm.SVC(probability=True)

clf.fit(X_train, y_train)


dump(clf, 'ModelSVM.joblib') 
clf_loaded = load('ModelSVM.joblib') 



In [69]:
roc_auc_score(y_test, clf_loaded.predict_proba(X_test)[:,1]) 

0.5151580347335117

In [68]:
roc_auc_score(y_test, clf_loaded.predict_proba(X_test)[:,1]) 

0.5151580347335117